In [1]:
import sys; sys.path.append('../'); sys.path.append('../hashing')
from hashing import LinearHash

from pathlib import Path
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

DATA_DIR = 'casos_lhash'

np.random.seed(88)

In [2]:
sorted(Path(DATA_DIR).iterdir())

[PosixPath('casos_lhash/ordered_blocks_creciente.npy'),
 PosixPath('casos_lhash/ordered_blocks_intercalado.npy'),
 PosixPath('casos_lhash/ordered_blocks_intercalado_rand.npy'),
 PosixPath('casos_lhash/ordered_step_1.npy'),
 PosixPath('casos_lhash/ordered_step_1_oversize.npy'),
 PosixPath('casos_lhash/ordered_step_3.npy'),
 PosixPath('casos_lhash/ordered_step_3_oversize.npy'),
 PosixPath('casos_lhash/ordered_step_quarter.npy'),
 PosixPath('casos_lhash/ordered_step_quarter_oversize.npy'),
 PosixPath('casos_lhash/ordered_step_random.npy'),
 PosixPath('casos_lhash/ordered_step_random_oversize.npy'),
 PosixPath('casos_lhash/repeated_3_1.npy'),
 PosixPath('casos_lhash/repeated_3_1_oversize.npy'),
 PosixPath('casos_lhash/repeated_3_3.npy'),
 PosixPath('casos_lhash/repeated_3_3_oversize.npy'),
 PosixPath('casos_lhash/repeated_3_quarter.npy'),
 PosixPath('casos_lhash/repeated_3_quarter_oversize.npy'),
 PosixPath('casos_lhash/repeated_quarter_1.npy'),
 PosixPath('casos_lhash/repeated_quarter_1_o

# Experimentos mejores y peores casos LinearHash

Sea $th$, el tamaño de tabla hash en cada instancia.

In [3]:
size_max = int(1e4)
sizes = [i for i in range(10, size_max, size_max//4)]

l_hashes = [LinearHash(size=s, update_size=True) for s in sizes]

df = pd.DataFrame(columns=['hash', 'original_size', 'final_size', 'caso',
                           'subcaso', 'caracteristica', 'oversize', 'tamano', 'tiempos', 'tiempo_total'])

## Secuencia de elementos ordenados

Se crean las siguientes variaciones de secuencias:

1. Secuencia ordenada creciente con paso de 1
2. Secuencia ordenada creciente con paso de 3
3. Secuencia ordenada creciente con paso de $\frac{th}{4}$
4. Secuencia ordenada creciente con paso de aleatorio entre 4 y $th$

5. Un cuarto creciente, otro decreciente y bis (bloques aleatorios)
6. 4 bloques crecientes independientes

Ademas para cada una de estas, existen dos tamaño: el tamaño de la tabla y 3 veces el tamaño de la tabla.

In [4]:
casos_ordenados_steps = list(Path(DATA_DIR).glob('ordered_step*'))
casos_ordenados_blocks = list(Path(DATA_DIR).glob('ordered_blocks*'))

arr_casos_ordenados_steps = [
    {caso.stem: np.load(caso, allow_pickle=True)} for caso in casos_ordenados_steps]

arr_casos_ordenados_blocks = [
    {caso.stem: np.load(caso, allow_pickle=True)} for caso in casos_ordenados_blocks]

casos_ordenados_steps, casos_ordenados_blocks

([PosixPath('casos_lhash/ordered_step_quarter_oversize.npy'),
  PosixPath('casos_lhash/ordered_step_3_oversize.npy'),
  PosixPath('casos_lhash/ordered_step_3.npy'),
  PosixPath('casos_lhash/ordered_step_1.npy'),
  PosixPath('casos_lhash/ordered_step_quarter.npy'),
  PosixPath('casos_lhash/ordered_step_1_oversize.npy'),
  PosixPath('casos_lhash/ordered_step_random.npy'),
  PosixPath('casos_lhash/ordered_step_random_oversize.npy')],
 [PosixPath('casos_lhash/ordered_blocks_intercalado.npy'),
  PosixPath('casos_lhash/ordered_blocks_creciente.npy'),
  PosixPath('casos_lhash/ordered_blocks_intercalado_rand.npy')])

### Steps

In [5]:
for caso in tqdm(arr_casos_ordenados_steps):
    tipo_caso, arrs = list(caso.items())[0]
    rows = []
    
    tipo_caso = tipo_caso.split('_')
    
    type_name = 'ordered'
    
    subtype = tipo_caso[1]
    caracteristica = tipo_caso[2
                              ]
    if tipo_caso[-1] == 'oversize':
        oversize = 1
    else:
        oversize = 0
    
    for arr in tqdm(arrs):
        l_hashes = [LinearHash(size=s, update_size=True) for s in sizes]
        for lh in tqdm(l_hashes):
            lh.run_experiment(arr)
            total_times, individual_times, original_size, final_size = lh.get_results()

            row = {
                'hash': 'lineal', 
                'original_size': original_size,
                'final_size' : final_size,
                'caso': type_name, 
                'subcaso' : subtype, 
                'caracteristica' : caracteristica, 
                'oversize' : oversize,
                'tamano': len(arr), 
                'tiempos': individual_times, 
                'tiempo_total': total_times
            }
            
        rows.append(row)
    df = df.append(pd.DataFrame(rows), ignore_index=True)

In [7]:
# df.to_csv('results.csv', index=False)

In [7]:
df = pd.read_csv('results.csv')

### Blocks

In [10]:
for caso in tqdm(arr_casos_ordenados_blocks):
    tipo_caso, arrs = list(caso.items())[0]
    rows = []
    
    tipo_caso = tipo_caso.split('_')
    
    type_name = 'ordered'
    
    subtype = tipo_caso[1]
    caracteristica = tipo_caso[2
                              ]
    if tipo_caso[-1] == 'oversize':
        oversize = 1
    else:
        oversize = 0
    
    for arr in tqdm(arrs[2:]):
        l_hashes = [LinearHash(size=s, update_size=True) for s in sizes]
        for lh in tqdm(l_hashes):
            lh.run_experiment(arr)
            total_times, individual_times, original_size, final_size = lh.get_results()

            row = {
                'hash': 'lineal', 
                'original_size': original_size,
                'final_size' : final_size,
                'caso': type_name, 
                'subcaso' : subtype, 
                'caracteristica' : caracteristica, 
                'oversize' : oversize,
                'tamano': len(arr), 
                'tiempos': individual_times, 
                'tiempo_total': total_times
            }
            
        rows.append(row)
    df = df.append(pd.DataFrame(rows), ignore_index=True)

In [39]:
df.to_csv('results.csv', index=False)

## Secuencia de elementos repetidos

Se crean las siguientes variaciones de secuencias:

1. Secuencia de 3 elementos repetidos con step de 1
2. Secuencia de 3 elementos repetidos con step de 3
3. Secuencia de 3 elementos repetidos con step de $\frac{th}{4}$
4. Secuencia de $\frac{th}{4}$ elementos repetidos con step aleatorio entre 4 y $th$

Ademas para cada una de estas, existen dos tamaño: el tamaño de la tabla y 3 veces el tamaño de la tabla.

In [10]:
df = pd.DataFrame(columns=['hash', 'original_size', 'final_size', 'caso',
                           'repeticiones', 'step_size', 'oversize', 'tamano', 'tiempos', 'tiempo_total'])

casos_repetidos_3 = list(Path(DATA_DIR).glob('repeated_3*'))
casos_repetidos_quarter = list(Path(DATA_DIR).glob('repeated_quarter*'))

arr_casos_repetidos_3 = [{caso.stem: np.load(caso, allow_pickle=True)} for caso in casos_repetidos_3]
arr_casos_repetidos_quarter = [{caso.stem: np.load(caso, allow_pickle=True)} for caso in casos_repetidos_quarter]

casos_repetidos_3, casos_repetidos_quarter

([PosixPath('casos_lhash/repeated_3_quarter_oversize.npy'),
  PosixPath('casos_lhash/repeated_3_1.npy'),
  PosixPath('casos_lhash/repeated_3_3.npy'),
  PosixPath('casos_lhash/repeated_3_1_oversize.npy'),
  PosixPath('casos_lhash/repeated_3_3_oversize.npy'),
  PosixPath('casos_lhash/repeated_3_quarter.npy')],
 [PosixPath('casos_lhash/repeated_quarter_1_oversize.npy'),
  PosixPath('casos_lhash/repeated_quarter_random.npy'),
  PosixPath('casos_lhash/repeated_quarter_random_oversize.npy'),
  PosixPath('casos_lhash/repeated_quarter_1.npy')])

In [11]:
for caso in tqdm(arr_casos_repetidos_3):
    tipo_caso, arrs = list(caso.items())[0]
    rows = []
    
    tipo_caso = tipo_caso.split('_')
    
    type_name = 'repeated'
    
    num_repeats = tipo_caso[1]
    step_size = tipo_caso[2]
    
    if tipo_caso[-1] == 'oversize':
        oversize = 1
    else:
        oversize = 0
    
    for arr in tqdm(arrs):
        l_hashes = [LinearHash(size=s, update_size=True) for s in sizes]
        for lh in tqdm(l_hashes):
            lh.run_experiment(arr)
            total_times, individual_times, original_size, final_size = lh.get_results()

            row = {
                'hash': 'lineal', 
                'original_size': original_size,
                'final_size' : final_size,
                'caso': type_name, 
                'repeticiones' : num_repeats, 
                'step_size' : step_size, 
                'oversize' : oversize,
                'tamano': len(arr), 
                'tiempos': individual_times, 
                'tiempo_total': total_times
            }
            
        rows.append(row)
    df = df.append(pd.DataFrame(rows), ignore_index=True)

In [13]:
df.to_csv('results_repeated.csv', index=False)

In [ ]:
# for caso in tqdm(arr_casos_repetidos_quarter):
#     tipo_caso, arrs = list(caso.items())[0]
#     rows = []
    
#     tipo_caso = tipo_caso.split('_')
    
#     type_name = 'repeated'
    
#     num_repeats = tipo_caso[1]
#     step_size = tipo_caso[2]
    
#     if tipo_caso[-1] == 'oversize':
#         oversize = 1
#     else:
#         oversize = 0
    
#     for arr in tqdm(arrs):
#         l_hashes = [LinearHash(size=s, update_size=True) for s in sizes]
#         for lh in tqdm(l_hashes):
#             lh.run_experiment(arr)
#             total_times, individual_times, original_size, final_size = lh.get_results()

#             row = {
#                 'hash': 'lineal', 
#                 'original_size': original_size,
#                 'final_size' : final_size,
#                 'caso': type_name, 
#                 'repeticiones' : num_repeats, 
#                 'step_size' : step_size, 
#                 'oversize' : oversize,
#                 'tamano': len(arr), 
#                 'tiempos': individual_times, 
#                 'tiempo_total': total_times
#             }
            
#         rows.append(row)
#     df = df.append(pd.DataFrame(rows), ignore_index=True)

## Secuencia de elementos aleatorias

Se crean las siguientes variaciones de secuencias:

1. Secuencias aleatorias

In [ ]:
df = pd.DataFrame(columns=['hash', 'original_size', 'final_size', 'caso',
                           'caracteristica', 'tamano', 'oversize', 'tiempos', 
                           'tiempo_total'])


casos_rand = list(Path(DATA_DIR).glob('random*'))
arr_casos_rand = [{caso.stem: np.load(caso, allow_pickle=True)} for caso in casos_rand]
casos_rand

In [ ]:
for caso in tqdm(arr_casos_rand):
    tipo_caso, arrs = list(caso.items())[0]
    rows = []
    
    tipo_caso = tipo_caso.split('_')
    
    type_name = 'random'
    
    caracteristica = tipo_caso[1]
    
    if tipo_caso[-1] == 'oversize':
        oversize = 1
    else:
        oversize = 0
        
    for arr in tqdm(arrs):
        d_hashes = [DoubleHash(size=s, update_size=True) for s in sizes]
        for dh in tqdm(d_hashes):
            dh.run_experiment(arr)
            total_times, individual_times, original_size, final_size = dh.get_results()

            row = {
                'hash': 'lineal', 
                'original_size': original_size,
                'final_size' : final_size,
                'caso': tipo_caso, 
                'caracteristica': caracteristica,
                'tamano': len(arr), 
                'oversize': oversize,
                'tiempos': individual_times, 
                'tiempo_total': total_times
            }
            
        rows.append(row)
    df = df.append(pd.DataFrame(rows), ignore_index=True)

In [ ]:
df.to_csv('results_double_random.csv', index=False)

In [ ]:
df